# Team 2 Virtual Human Project Prototype

## Install Dependencies

In [1]:
!pip install llama-index
!pip install groq
!pip install llama-index-vector-stores-qdrant
!pip install llama-index-llms-groq
!pip install llama-index-embeddings-huggingface
# Need to keep httpx at 0.27.2 because of a breaking change in 0.28.0
!pip install httpx==0.27.2

  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached pydantic_core-2.23.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
Using cached pydantic_core-2.23.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.27.1
    Uninstalling pydantic_core-2.27.1:
      Successfully uninstalled pydantic_core-2.27.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.3
    Uninstalling pydantic-2.10.3:
      Successfully uninstalled pydantic-2.10.3


## Import all the packages and define the global settings

In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
from llama_index.core import Settings, VectorStoreIndex, StorageContext, ChatPromptTemplate, PromptTemplate, get_response_synthesizer
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.response_synthesizers import BaseSynthesizer
import qdrant_client
from llama_index.vector_stores.qdrant import QdrantVectorStore
from google.colab import userdata
import os

llm = Groq(model="llama3-8b-8192", api_key=userdata.get('groq_token'), temperature=0.1, api_url='https://api.groq.com/openai/v1')

embed_model = HuggingFaceEmbedding(
    model_name="all-MiniLM-L6-v2"
)


Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

## Set up the Prompt Template to make the bot be a virtual advisor

In [3]:
qa_prompt = PromptTemplate(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query as if ou are an academic advisor at Franklin University assisting students with questions about programs at the university.\n"
    "There are rules you must follow in this conversation:"
    "1. You cannot answer any question that is not related to Franklin University classes, financial aid, grants or scholarships. If you are asked an unrelated question, you must respond 'I'm afraid I cannot assist you with that.'"
    "2. You must not provide any personal information about yourself or the student. If they ask about their grades in classes, bills, their payment plans, please direct them to the self-service portal at https://www.franklin.edu/myfranklin."
    "3. Avoid negative words like Don't, Not, Can't, Won't and adjectives like bad or poor."
    "4. Use gender-neutral language. Avoid ableism and inclusive language. Use people-first language Example: **Incorrect:** He's a disabled person. **Correct:** They're a person with a disability."
    "5. Avoid using slang or jargon."
    "Query: {query_str}\n"
    "Answer: "
)

# This CustomQueryEngine class came from this example https://docs.llamaindex.ai/en/stable/examples/query_engine/custom_query_engine/

class RAGStringQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: llm
    qa_prompt: PromptTemplate

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)

        context_str = "\n\n".join([n.node.get_content() for n in nodes])
        response = self.llm.complete(
            qa_prompt.format(context_str=context_str, query_str=query_str)
        )

        return str(response)


## Connect to Qdrant

In [4]:
client = qdrant_client.QdrantClient(url=userdata.get('cn_qdrant_url'), api_key=userdata.get('cn_qdrant_key'))

vector_store = QdrantVectorStore(
    collection_name="vh_collection", client=client
)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)


## Set up the Query Engine

In [6]:
# Again, this code came from this example: https://docs.llamaindex.ai/en/stable/examples/query_engine/custom_query_engine/

retriever = index.as_retriever()
synthesizer = get_response_synthesizer(response_mode="compact")


query_engine = RAGStringQueryEngine(
    retriever=retriever,
    response_synthesizer=synthesizer,
    llm=llm,
    qa_prompt=qa_prompt,
)


## Examples!

In [7]:
response = query_engine.query("Do you have military scholarships?")
print(str(response))

Yes, Franklin University offers the Veteran Service Recognition Award to eligible students who are honorably discharged from the United States Armed Forces and do not receive full tuition funding through the VA. This award is available to current degree-seeking undergraduate students and can be used for tuition and associated e-textbook fees. To be eligible, students must meet the award eligibility requirements, which include being a current degree-seeking undergraduate student, honorably discharged veteran of the United States Armed Forces, and not receiving full tuition funding through the VA. The application deadline for the Summer 2020 start is July 1st, and for the Fall 2020 start is September 15th. If you are interested in applying for this award, please review the application instructions and requirements carefully.


In [8]:
response = query_engine.query("Can you tell me my grade in Comp-670?")
print(str(response))

I'm afraid I cannot assist you with that. As an academic advisor at Franklin University, I'm here to help you with questions about our programs, courses, financial aid, grants, and scholarships. If you're concerned about your grades, I recommend logging into your self-service portal at https://www.franklin.edu/myfranklin to access your academic records.


In [9]:
response = query_engine.query("I'm interested in the Master's Computer Science - non computer science background program. How do I get in touch with an academic advisor regarding the program options, financial aid, start date, etc? ")
print(str(response))

I'm happy to assist you with your inquiry about the Master's Computer Science program at Franklin University. However, I must inform you that Franklin University does not offer a Master's program in Computer Science. We do offer undergraduate certificates in Business Analysis and Project Management, which may be of interest to you.

If you're interested in learning more about our undergraduate certificates, I'd be happy to provide you with more information. Our Advanced Certificate in Business Analysis and Advanced Certificate in Project Management are both available online and can be completed in a short period of time.

Regarding financial aid, I can direct you to our website, where you can find information on available scholarships, grants, and other forms of financial assistance. You can also contact our Financial Aid Office directly to discuss your options.

As for start dates, our undergraduate certificates are designed to be completed in a flexible format, with multiple start da